In [ ]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

!pip install gdelt

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://securit

In [ ]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

Fetching Data

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


I want to sort the data into two types, ones with lower than 2% annual GDP growth or negative annual GDP growth and normally expected growth from 2018-2020, the downloaded file is from the OECD Data https://data.oecd.org/chart/6JB7where GDP is expressed in millions of $USD. I then calculate the GDP growth rate using the gdp growth rate formula.

In [ ]:
import csv
import pandas as pd

url = "https://raw.githubusercontent.com/reed160/DATA301-PROJECT-OECD/main/2018-2020.csv"
df = pd.read_csv(url,index_col=0)
df.to_csv("2018-2020.csv",encoding='utf-8',index=True)

with open('2018-2020.csv', newline='') as csvfile:
  gdp_growth = dict()
  reader = csv.DictReader(csvfile)
  check_list = []
  for row in reader:
    country_name = row["LOCATION"]
    if row['TIME'] == "2018":
        #assigning the intial amount of GDP in 2018
        gdp_growth[country_name] = row['Value']
    elif row['TIME'] ==  "2020" and country_name in gdp_growth.keys():
      #assigning the previous gdp to a variable
      prev_gdp = gdp_growth[country_name]
      #calculating the gdp growth rate
      gdp_growth[country_name] = ((float(row['Value']) - float(prev_gdp)) / (float(prev_gdp) * 100)) * 10000
      #the country is added to the check list of countries that have a value for 2020
      check_list.append(country_name)

#deleting items that dont have data for 2020
missing_data = gdp_growth.keys() - check_list
for item in missing_data:
  del gdp_growth[item]

print(gdp_growth)

low_gdp = []
positive_gdp = []
for item in gdp_growth.keys():
  #dividing gdp growth by the number of years and checking if its lower than average
  if gdp_growth[item]/3 < 2:
    low_gdp.append(item)
  else:
    positive_gdp.append(item)
#removes the two European groups as they differ from the CAMEO codes and the acculumative gdp of several countries can be unreliable and difficult
low_gdp = [e for e in low_gdp if e not in ("EA19", "EU27_2020")]
#removes these codes as they differ from the CAMEO codes
positive_gdp = [e for e in positive_gdp if e not in ("OECD", "OECDE", "ROU")]
#adds back the code for romania
positive_gdp.append("ROM")
print(low_gdp)
print(positive_gdp)


{'AUS': 7.997634917071514, 'AUT': -1.3927263624975728, 'BEL': 2.054270951059482, 'CAN': -4.205927168623209, 'CZE': 1.79510737510515, 'DNK': 5.461763661698918, 'FIN': 2.997135762644858, 'FRA': 1.3081158717136008, 'DEU': -0.4021190465621561, 'GRC': -5.918923600333928, 'HUN': 3.994490094096228, 'ISL': -3.039246292294462, 'IRL': 13.085515410442799, 'ITA': -4.006042975661709, 'JPN': -0.4720532497208101, 'KOR': 5.579762229406596, 'LUX': 4.476535228138291, 'MEX': -5.073862412319773, 'NLD': 3.738999086924107, 'NZL': 9.336097499195075, 'NOR': -9.120316718172088, 'POL': 7.658411474372657, 'PRT': -2.0252677143260525, 'SVK': 0.6767322520442415, 'ESP': -6.102960128098696, 'SWE': 4.685507125177635, 'CHE': 1.5560090811937615, 'TUR': -0.23019972296126742, 'GBR': -1.2865885303752427, 'USA': 1.7858633043179526, 'CHL': -0.33083731162455887, 'CHN': 11.661259175801025, 'COL': 1.2644764463245282, 'EST': 4.43996172771676, 'IDN': 5.929384995989764, 'ISR': 2.657330388742002, 'RUS': 3.2007664659588624, 'SVN': 3

Fetching the data:

In [ ]:
#used to store files locally when testing
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()


# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)

def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date) #not updating at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
    except:
        print("Error occurred")

# pulling the data from gdelt into multi files, before the stock market collapse of 2020 and after
dates_before = [get_filename(x) for x in pd.date_range('2019 Dec 1','2020 Feb 20')]
dates_after = [get_filename(x) for x in pd.date_range('2020 Feb 21','2020 April 1')]

results = list(e.map(intofile,dates_before+dates_after))

/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = eventWork(self.download_list)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = eventWork(self.download_list)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = eventWork(self.download_list)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = eventWork(self.download_list)
/usr/local/lib/python3.7/dist-packages/gdelt/base.py:617: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = 

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#process these into RDDs
data_before = sqlContext.read.option("header", "true").csv(dates_before)
data_after = sqlContext.read.option("header", "true").csv(dates_after)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
#seperate the two possible events Im looking at, providing aid and the intent to provide aid
provide_aid = ["070", "071", "072", "073", "074", "075"]
express_intent_to_provide_aid = ['033', "0331", "0332", "0333", "0334"]

#checking the event code and returning the matching event ID
def check_if_economic_aid(code):
  return("Aid provided" if code in provide_aid else
         "Intent expressed" if code in express_intent_to_provide_aid else "Other")
 
  

Answering the research question of if  countries that experienced negative/low long term gdp growth provided less international aid than countries that experienced positive long term gdp growth.

In [ ]:
import numpy as np
import time
start_time = time.time()

def get_economic_aid(data, countries):
  #map the rdd to country and event, with a weighting of 1
  country_event_types = data.rdd.map(lambda row: ((row['Actor1CountryCode'], check_if_economic_aid(row['EventCode'])), 1))
  #filter the country events such that there are no None values
  country_event_types = country_event_types.filter(lambda x: x[0][0] is not None)
  #dbg(country_event_types)

  #reduce by the key tuple
  summary = country_event_types.reduceByKey(lambda a, b: a+b)
  #filter the summary
  country_summary = summary.filter(lambda kv: kv[0][0] in countries)
  return {k:v for k,v in country_summary.collect()}

#the function which creates a vector for a country 
def vector(result, country):
  #try get the result for a country and event, set the value to 0 if there is no value
  try:
    aid_provided_result = result[(country, 'Aid provided')]
  except:
    aid_provided_result = 0
  try:
    intent_expressed_result = result[(country, 'Intent expressed')]
  except:
    intent_expressed_result = 0
  return np.array([aid_provided_result,
          intent_expressed_result])

#calculate the similarity between two seperate vectors using the Cosine distance
def similarity(vector1, vector2):
  return vector1.dot(vector2) / (np.linalg.norm(vector1)*np.linalg.norm(vector2))

#create the summary for the data that has a low level of gdp
summary_before_low = get_economic_aid(data_before, low_gdp)
summary_after_low = get_economic_aid(data_after, low_gdp)
#create the vectors for the countries with a low level of gdp
before_low = {a:vector(summary_before_low, a) for a in low_gdp}
after_low = {a:vector(summary_after_low, a) for a in low_gdp}

#create the summary for the data that has a high level of gdp
summary_before_high = get_economic_aid(data_before, positive_gdp)
summary_after_high = get_economic_aid(data_after, positive_gdp)
#create the vectors for the countries with a high level of gdp
before_high = {a:vector(summary_before_high, a) for a in positive_gdp}
after_high = {a:vector(summary_after_high, a) for a in positive_gdp}

#create a total vector for both the high level gdp vectors and a total vector for the low level gdp vector
before_low_total = after_low_total = before_high_total = after_high_total = np.array([0,0])

print("Countries with a low level of gdp growth:")
for x in low_gdp:
  try:
    #print the vectors before and after the stock market collapse
    print(x, "Vectors before->after", before_low[x], "->", after_low[x])
    #add the vectors to the total
    before_low_total = before_low_total + before_low[x]
    after_low_total = after_low_total + after_low[x]
  except:
    pass

print("Countries with a high level of gdp growth:")
for x in positive_gdp:
  try:
    #print the vectors before and after the stock market collapse
    print(x, "Vectors before->after", before_high[x], "->", after_high[x])
    #add the vectors to the total
    before_high_total = before_high_total + before_high[x]
    after_high_total = after_high_total + after_high[x]
  except:
    pass
#compare the similarities between high gdp growth and low gdp growth before the stock market crash then after the stock market crash
before_similarity = similarity(before_high_total, before_low_total)
after_similarity = similarity(after_high_total, after_low_total)

print("Countries with low GDP growth", "similiarity before->after", before_similarity, "->", after_similarity)
print(after_similarity - before_similarity, "Low GDP growth")
print("The program took {} to run".format(time.time()-start_time))

Countries with a low level of gdp growth:
AUT Vectors before->after [3 0] -> [1 0]
BEL Vectors before->after [9 1] -> [3 0]
CAN Vectors before->after [83 13] -> [49  6]
CZE Vectors before->after [0 0] -> [1 0]
DNK Vectors before->after [3 0] -> [2 0]
FIN Vectors before->after [1 0] -> [0 0]
FRA Vectors before->after [16 16] -> [3 1]
DEU Vectors before->after [35  2] -> [8 2]
GRC Vectors before->after [11  0] -> [11  0]
HUN Vectors before->after [6 1] -> [0 0]
ISL Vectors before->after [0 0] -> [0 0]
ITA Vectors before->after [10  0] -> [18  3]
JPN Vectors before->after [10  5] -> [6 0]
KOR Vectors before->after [3 0] -> [9 0]
LUX Vectors before->after [2 0] -> [0 0]
MEX Vectors before->after [17  1] -> [11  0]
NLD Vectors before->after [4 0] -> [5 0]
NOR Vectors before->after [4 0] -> [2 0]
PRT Vectors before->after [0 0] -> [1 0]
SVK Vectors before->after [1 0] -> [2 0]
ESP Vectors before->after [3 0] -> [5 0]
SWE Vectors before->after [4 0] -> [1 0]
CHE Vectors before->after [12  0] 